In [1]:
###IMPORTOWANIE BIBLIOTEK

import pandas as pd
import os, os.path
import sqlite3
import Jakub_Kępka_PD2

In [2]:
# Importowanie danych z plikó csv.

Posts = pd.read_csv("Posts.csv")
PostLinks = pd.read_csv("PostLinks.csv")
Users = pd.read_csv("Users.csv")
Comments = pd.read_csv("Comments.csv")

In [3]:
### FUNKCJE

# Sortujemy dataframe przez pierwsze dwie kolumny.
# Zwracamy posortowany dataframe.
def SortDataFrame(df):
    sorted_df = df.sort_values(by=[df.columns[0], df.columns[1]])
    return sorted_df

# Funkcja sprawdza czy 2 dataframy są ze sobą tożsame.
# Pierwsze sortuje obie tabele, następnie zmienia numeracje, aby się nie różniły.
# Porównujemy metodą equals(), jeżeli są tożsame wypisujemy że są identyczne.
# W przeciwynym przypadku wypisujemy zawartość 2 tabeli, dzięki czemu możemy je porównać.
def CheckIfCorrect(sql_ans, python_ans, number):

    sql_ans = SortDataFrame(sql_ans)
    python_ans = SortDataFrame(python_ans)
    
    sql_ans = sql_ans.reset_index(drop=True)
    python_ans = python_ans.reset_index(drop=True)

    if sql_ans.equals(python_ans):
        print("DataFrames " + str(number) + " są identyczne.")

        #print("SQL:")
        #print(sql_ans)

        #print("Python:")
        #print(python_ans)
    else:
        print("DataFrames " + str(number) + " różnią się.")

        print("SQL:")
        print(sql_ans)

        print("Python:")
        print(python_ans)


In [4]:

import importlib
importlib.reload(Jakub_Kępka_PD2)

# sciezka dostępu do bazy danych:
baza = os.path.join('baza2.db')
if os.path.isfile(baza): 
    os.remove(baza) 

conn = sqlite3.connect(baza) # połączenie do bazy danych

# Importowanie tabeli do bazy danych.
Posts.to_sql("Posts", conn) 
PostLinks.to_sql("PostLinks", conn) 
Users.to_sql("Users", conn) 
Comments.to_sql("Comments", conn) 


# Dla każdego z 5 zapytań wywołuję zapytanie SQL oraz własnoręcznie zaimplementowana metodę.
# Następnie sprawdzam poprawność wyników przy pomocy wyżej zaimplementowanej metody CheckIfCorrect().

sql_ans_1 = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn) 

python_ans_1 = Jakub_Kępka_PD2.solution_1(Posts, Users)

CheckIfCorrect(sql_ans_1, python_ans_1, 1)

sql_ans_2 = pd.read_sql_query("""
SELECT Posts.Title, RelatedTab.NumLinks
FROM
(
SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks
GROUP BY RelatedPostId
) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
""", conn)

python_ans_2 = Jakub_Kępka_PD2.solution_2(Posts, PostLinks)

CheckIfCorrect(sql_ans_2, python_ans_2, 2)

sql_ans_3 = pd.read_sql_query("""
SELECT Title, CommentCount, ViewCount, CommentsTotalScore,
DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10

""", conn)

python_ans_3 = Jakub_Kępka_PD2.solution_3(Comments, Posts, Users)

CheckIfCorrect(sql_ans_3, python_ans_3, 3)

sql_ans_4 = pd.read_sql_query("""
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location,
Reputation, UpVotes, DownVotes
FROM (
SELECT *
FROM (
SELECT COUNT(*) as AnswersNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 2
GROUP BY OwnerUserId
) AS Answers
JOIN
(
SELECT COUNT(*) as QuestionsNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
ON Answers.OwnerUserId = Questions.OwnerUserId
WHERE AnswersNumber > QuestionsNumber
ORDER BY AnswersNumber DESC
LIMIT 5
) AS PostsCounts
JOIN Users ON PostsCounts.OwnerUserId = Users.Id
                              
""", conn)

python_ans_4 = Jakub_Kępka_PD2.solution_4(Posts, Users)

CheckIfCorrect(sql_ans_4, python_ans_4, 4)

sql_ans_5 = pd.read_sql_query("""
SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10                         
""", conn)

python_ans_5 = Jakub_Kępka_PD2.solution_5(Posts,Users)

CheckIfCorrect(sql_ans_5, python_ans_5, 5)

print("Koniec")

conn.close()


DataFrames 1 są identyczne.
DataFrames 2 są identyczne.
DataFrames 3 są identyczne.
DataFrames 4 są identyczne.
DataFrames 5 są identyczne.
Koniec
